In [1]:
from load_data import *
from partition import *
from desaturation_fev1 import *
from normalise import *
from exacerbation_periods import *
from smooth import *
from plot_helpers import *
from biology import *
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go


#Settings
datadir = "../../../../SmartCareData/"
plotsdir="../../../../PlotsSmartcare/O2_FEV1/"

In [2]:
# Process O2-FEV1 data
O2_FEV1 = create_O2_FEV1_df(datadir)
# O2_FEV1 = remove_all_measurements_with_ID_216(O2_FEV1)

# Compute FEV1 % Predicted
O2_FEV1['FEV1 % Predicted'] = O2_FEV1.apply(lambda x: x["FEV1"] / x["Predicted FEV1"] * 100, axis=1)

# Partition FEV1 % Predicted in 3 groups
O2_FEV1["FEV1 % Predicted Group"] = partition_in_n_equal_groups(O2_FEV1['FEV1 % Predicted'],3)

# What are we analysing?
O2_FEV1.head()

/Users/tristan.trebaol/Desktop/PhD/Code/old_phd/src/O2-FEV1 analysis/load_data.py:14: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  measurements = pd.read_csv(datadir + "mydata.csv").rename(columns={"FEV 1": "FEV1"})
/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/Applications/anaconda3/envs/phd/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


Removed 11 measurements where O2 Sat > 100%, kept 123136 measurements
O2 Saturation contains 13418 measurements
FEV1 contains 10418 measurements
Removed 4684 rows with O2_FEV1 inner join, kept 68% of measurements (10100)
Removed 1008 duplicates, 9092 measurements left


,ID,Date recorded,O2 Saturation,FEV1,Age,Sex,Height,Weight,Predicted FEV1,FEV1 Set As,Pulmonary Exacerbation,Less Exacerbation,Transplant Recipients,Date Last PE Start,Date Last PE Stop,Comments,FEV1 % Predicted,FEV1 % Predicted Group
0,79,2015-09-10,95.0,1.53,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,40.691489,<42.7
1,79,2015-09-11,97.0,1.68,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,44.680851,[42.7;66.0[
2,79,2015-09-12,97.0,1.48,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,39.361702,<42.7
3,79,2015-08-13,95.0,1.63,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,43.351064,[42.7;66.0[
4,79,2015-11-07,94.0,1.52,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,40.425532,<42.7


# Analysis of the erroneously dropped data

In [3]:
full = merge_pred_ex_labels_to(O2_FEV1, get_pred_ex_labels(datadir))
print("{} IDs have measurements in exacerbation periods".format(full.ID[full["Is Exacerbated"]==True].nunique()))

nonan = full.dropna()
# print("{} measurements kept after erroneously dropping nans".format(nonan.shape[0]))

# Select only rows where isna is True
nan = full[full.isna().any(axis=1)]
full.head()

Exacerbated labels data from the predictive classifier has 14452 entries (1445 exacerbated, 13006 not exacerbated measurements, 1 NaN)
** Inner merge of O2_FEV1 and exacerbated labels on 'ID' and 'Date recorded' **
Data has now 7577 entries and 98 IDs (initially 9092 & 141 in O2_FEV1, 14452 in pred_ex_labels)
51 IDs have measurements in exacerbation periods


,ID,Date recorded,O2 Saturation,FEV1,Age,Sex,Height,Weight,Predicted FEV1,FEV1 Set As,Pulmonary Exacerbation,Less Exacerbation,Transplant Recipients,Date Last PE Start,Date Last PE Stop,Comments,FEV1 % Predicted,FEV1 % Predicted Group,Is Exacerbated
0,79,2015-08-13,95.0,1.63,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,43.351064,[42.7;66.0[,True
1,79,2015-11-07,94.0,1.52,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,40.425532,<42.7,False
2,79,2015-10-14,95.0,1.66,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,44.148936,[42.7;66.0[,False
3,79,2015-10-21,93.0,1.64,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,43.617021,[42.7;66.0[,False
4,79,2015-12-11,95.0,1.50,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,39.893617,<42.7,False


In [4]:
# What's nan?
nan.head()

,ID,Date recorded,O2 Saturation,FEV1,Age,Sex,Height,Weight,Predicted FEV1,FEV1 Set As,Pulmonary Exacerbation,Less Exacerbation,Transplant Recipients,Date Last PE Start,Date Last PE Stop,Comments,FEV1 % Predicted,FEV1 % Predicted Group,Is Exacerbated
0,79,2015-08-13,95.0,1.63,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,43.351064,[42.7;66.0[,True
1,79,2015-11-07,94.0,1.52,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,40.425532,<42.7,False
2,79,2015-10-14,95.0,1.66,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,44.148936,[42.7;66.0[,False
3,79,2015-10-21,93.0,1.64,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,43.617021,[42.7;66.0[,False
4,79,2015-12-11,95.0,1.50,27,Male,163.0,54.7,3.76,3.8,Yes,No,No,2015-07-13,2015-08-06 00:00:00,NaN,39.893617,<42.7,False


In [5]:
# What's not nan?
nonan.head()

,ID,Date recorded,O2 Saturation,FEV1,Age,Sex,Height,Weight,Predicted FEV1,FEV1 Set As,Pulmonary Exacerbation,Less Exacerbation,Transplant Recipients,Date Last PE Start,Date Last PE Stop,Comments,FEV1 % Predicted,FEV1 % Predicted Group,Is Exacerbated
310,30,2015-12-10,95.0,1.06,33,Female,155.4,57.5,2.7,2.7,Yes,No,No,2015-11-23,2015-12-07 00:00:00,Advised by medical team to stop study as too a...,39.259259,<42.7,False
311,30,2015-12-14,98.0,1.02,33,Female,155.4,57.5,2.7,2.7,Yes,No,No,2015-11-23,2015-12-07 00:00:00,Advised by medical team to stop study as too a...,37.777778,<42.7,False
312,30,2015-12-09,95.0,0.99,33,Female,155.4,57.5,2.7,2.7,Yes,No,No,2015-11-23,2015-12-07 00:00:00,Advised by medical team to stop study as too a...,36.666667,<42.7,False
313,30,2015-12-31,97.0,1.00,33,Female,155.4,57.5,2.7,2.7,Yes,No,No,2015-11-23,2015-12-07 00:00:00,Advised by medical team to stop study as too a...,37.037037,<42.7,False
314,30,2015-12-13,94.0,1.04,33,Female,155.4,57.5,2.7,2.7,Yes,No,No,2015-11-23,2015-12-07 00:00:00,Advised by medical team to stop study as too a...,38.518519,<42.7,False


In [6]:
# We erronesouly removed rows where comments was NaN. There's no biological reason for this. Let's still have a look at where does the difference lie.

# Scatter plot of nan with x Date recorded, y O2 saturation, color Is Exacerbated with go library
x_col='Date recorded'
fig = go.Figure()
fig.add_trace(go.Scatter(x=nan[x_col], y=nan['O2 Saturation'], mode='markers', marker_color=nan['Is Exacerbated'].apply(lambda l: get_ex_color() if l else get_stable_color())))
# Put x axis range to min max of Date recorded of full
fig.update_xaxes(range=[full[x_col].min(), full[x_col].max()])
fig.show()
# 
fig = go.Figure()
fig.add_trace(go.Scatter(x=nonan[x_col], y=nonan['O2 Saturation'], mode='markers', marker_color=nonan['Is Exacerbated'].apply(lambda l: get_ex_color() if l else get_stable_color())))
fig.update_xaxes(range=[full[x_col].min(), full[x_col].max()])
fig.show()

In [7]:
print("Number of IDs after dropping rows with NaN comments: {}".format(nonan.ID.nunique()))
print("Number of IDs after correction: {}".format(full.ID.nunique()))

# List IDs present in nan but not in nonan
nan_ids = nan.ID.unique()
nonan_ids = nonan.ID.unique()
print("IDs present in nan but not in nonan: {}".format([l for l in nan_ids if l not in nonan_ids]))

Number of IDs after dropping rows with NaN comments: 36
Number of IDs after correction: 98
IDs present in nan but not in nonan: ['79', '23', '138', '24', '80', '81', '137', '139', '140', '82', '92', '99', '100', '93', '102', '113', '28', '114', '123', '115', '126', '127', '129', '128', '130', '131', '233', '78', '132', '133', '134', '136', '241', '59', '189', '187', '188', '231', '186', '141', '143', '191', '194', '195', '144', '193', '151', '197', '196', '234', '153', '66', '236', '171', '173', '175', '176', '172', '200', '179', '178', '229']


In [17]:
# Compare the displots before and after removing the rows with nan
def distplot(df, fig, row, col):
  fig.add_trace(go.Histogram(x=df['O2 Saturation'][df['Is Exacerbated']==True], histnorm='probability', marker=dict(color=get_ex_color()), name='Exacerbated'), row=row, col=col)
  fig.add_trace(go.Histogram(x=df['O2 Saturation'][df['Is Exacerbated']==False], histnorm='probability', marker=dict(color=get_stable_color()), name='Stable'), row=row, col=col)
  # Add vertical gridlines x axis
  fig.update_xaxes(showgrid=True, gridwidth=1, row=row, col=col)
  fig.update_layout(barmode="overlay")
  return -1

# Create figure with 3 rows and 1 column
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, x_title='O2 saturation (%)', y_title='Probability', 
  subplot_titles=("{} columns erroneously removed".format(nan.shape[0]),"After correction"))

distplot(nonan, fig, 1, 1)
distplot(full, fig, 2, 1)

# distplot(full[~full.ID.isin(['141', '59', '241', '194'])])
# distplot(full[~full.ID.isin(['79', '23', '138', '24', '80', '81', '137', '139', '140', '82', '92', '99', '100', '93', '102', '113', '28', '114', '123', '115', '126', 
#   '127', '129', '128', '130', '131', '233', '78', '132', '133', '134', '136', '241', '59', '189', '187', '188', '231', '186', '141', '143', '191', '194', '195', '144', 
#   '193', '151', '197', '196', '234', '153', '66', '236', '171', '173', '175', '176', '172', '200', '179', '178', '229'])],
#   fig, 3, 1)

fig.update_layout(width=800, height=600)
fig.show()

In [18]:
# We'd like to explain the difference in the distribution of O2 saturation in the Exacerbated and Stable groups before and after the removal of the rows with nan.
# Changes in O2 Saturation's distribution are much stronger in the Exacerbated group than in the Stable. 
# Let's have a look at the Exacerbated group only.

# Scatter plot nan with x Date recorded, y O2 saturation, color ID with px library
nan_ex = nan[nan['Is Exacerbated']==True]
nonan_ex = nonan[nonan['Is Exacerbated']==True]
full_ex = full[full['Is Exacerbated']==True]
print(nan_ex.shape)
print(nonan_ex.shape)

def set_axes(fig):
  # Set y axis range to min max of O2 saturation of full
  fig.update_yaxes(range=[full['O2 Saturation'].min()-1, full['O2 Saturation'].max()+1])
  # Set x axis range to min max of Date recorded of full
  fig.update_xaxes(range=[full[x_col].min(), full[x_col].max()])
  return -1

fig = px.scatter(nan_ex, x=x_col, y='O2 Saturation', color='ID')
# Add line of O2 saturation 95
fig.add_shape(type="line", x0=full[x_col].min(), y0=95, x1=full[x_col].max(), y1=95, line=dict(color="Black", width=0.5))
set_axes(fig)
fig.update_layout(title='Measurements in exacerbated period - With values erroneously dropped ({} points)'.format(nan_ex.shape[0]))
fig.show()

fig = px.scatter(nonan_ex, x=x_col, y='O2 Saturation', color='ID')
set_axes(fig)
fig.add_shape(type="line", x0=full[x_col].min(), y0=95, x1=full[x_col].max(), y1=95, line=dict(color="Black", width=0.5))
fig.update_layout(title='Measurements in exacerbated period - After erroneously dropping ({} points)'.format(nonan_ex.shape[0]))
fig.show()

fig = px.scatter(full_ex, x=x_col, y='O2 Saturation', color='ID')
set_axes(fig)
fig.add_shape(type="line", x0=full[x_col].min(), y0=95, x1=full[x_col].max(), y1=95, line=dict(color="Black", width=0.5))
fig.update_layout(title='Measurements in exacerbated period - After correction ({} points)'.format(full_ex.shape[0]))
fig.show()

(494, 19)
(210, 19)


In [19]:
# There is a signal within the population for the measurements done during exacerbation periods 
# Let's try to group individuals that have different behaviours during exacerbation periods. 

# Plot patients by interquartile range of O2 saturation during exacerbation periods

# Plot a boxplot of the O2 saturation values by ID
fig = px.box(full_ex, x='ID', y='O2 Saturation')
# Sort by mean O2 saturation
mean_o2 = full_ex.groupby('ID')['O2 Saturation'].mean().sort_values()
fig.update_layout(xaxis={'categoryorder':'array', 'categoryarray':mean_o2.index}) 
# Add title: Individual O2 saturation during exacerbation periods (number of IDs)
fig.update_layout(title='Individual O2 saturation during exacerbation periods ({} IDs)'.format(full_ex['ID'].nunique())) 
fig.show()

In [27]:
# 3 groups of patients can be identified:
# iqr < 94, i.e. max iqr < 94
# iqr in [94, 96], i.e. min iq >= 94 and max iqr <= 96
# iqr > 96, i.e. min iqr > 96

# Let's plot distribution of O2 saturation for measurements made in ex and stable periods for each group

# Compute the interquartile range of O2 saturation during exacerbation periods
def get_upper_iqr_val(series):
  return series.quantile(0.75)
def get_lower_iqr_val(series):
  return series.quantile(0.25)

# Create a DataFrame with the interquartile range of O2 saturation during exacerbation periods for each patient
df_iqr = full[full['Is Exacerbated']==True].groupby('ID')['O2 Saturation'].agg([get_upper_iqr_val, get_lower_iqr_val]).reset_index()

def set_o2_sat_iqr(df):
  # Initialize O2 Sat IQR column
  df['O2 Sat IQR'] = ''
  # Set O2 Sat IQR to <94 if max iqr < 94
  df.loc[df['get_upper_iqr_val'] < 94, 'O2 Sat IQR'] = '<94'
  # Set O2 Sat IQR to [94,96] if min iqr >= 94 and max iqr <= 96
  df.loc[(df['get_lower_iqr_val'] >= 94) & (df['get_upper_iqr_val'] <= 96), 'O2 Sat IQR'] = '[94,96]'
  # Set O2 Sat IQR to >96 if min iqr > 96
  df.loc[df['get_lower_iqr_val'] > 96, 'O2 Sat IQR'] = '>96'
  return df

df_iqr = set_o2_sat_iqr(df_iqr)
df_iqr

# Merge df_iqr 'O2 Sat IQR' with full on ID
full_iqr = full.merge(df_iqr[['ID', 'O2 Sat IQR']], on='ID', how='inner')

full_iqr['O2 Sat IQR'].value_counts()

           1604
[94,96]    1211
>96         516
<94         475
Name: O2 Sat IQR, dtype: int64

In [32]:
# Create figure with 3 rows and 1 column
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, x_title='O2 saturation (%)', y_title='Probability', 
  subplot_titles=(
    'iqr <94 ({} IDs)'.format(full_iqr[full_iqr['O2 Sat IQR']=='<94']['ID'].nunique()), 
  'iqr in [94,96] ({} IDs)'.format(full_iqr[full_iqr['O2 Sat IQR']=='[94,96]']['ID'].nunique()),
  'iqr >96 ({} IDs)'.format(full_iqr[full_iqr['O2 Sat IQR']=='>96']['ID'].nunique())
  )
  )

distplot(full_iqr[full_iqr['O2 Sat IQR']=='<94'], fig, 1, 1)
distplot(full_iqr[full_iqr['O2 Sat IQR']=='[94,96]'], fig, 2, 1)
distplot(full_iqr[full_iqr['O2 Sat IQR']=='>96'], fig, 3, 1)

fig.update_layout(width=800, height=800, title_text='Distribution of O2 saturation during exacerbation and stable periods ({} IDs)'.format(full_iqr['ID'].nunique()))
fig.show()